In [1]:
#Chunk 1: Import libraries

import zipfile
import os
import pandas as pd
from datetime import datetime
import shutil
import sys

In [2]:
#Chunk 2: Initialize
# Enter Park Code
park_name = input("Enter the Park Code: ").strip()

# Set which calendar to use
tool_mode = input("Do you want the tool to be based on 1. the current date or 2. a custom date range: ").strip()

# Debugging: Check the value of tool_mode
print(f"Tool mode entered: {tool_mode}")

if tool_mode == '1':  # Compare as string
    current_date_mode = input("Show 1. all active schedules or 2. all active and future schedules: ").strip()
    if current_date_mode == '1' or current_date_mode == '2':
        print(f"Current date mode entered: {current_date_mode}")
    else:
        print("Invalid option selected for tool mode.")
        sys.exit()
elif tool_mode == '2':  # Compare as string
    custom_start = input("Define custom start date as YYYYMMDD: ").strip()
    custom_end = input("Define custom end date as YYYYMMDD: ").strip()
    print(f"Creating GTFS for dates between {custom_start} and {custom_end}")
    custom_start = pd.to_datetime(custom_start, format='%Y%m%d')
    custom_end = pd.to_datetime(custom_end, format='%Y%m%d')
else:
    print("Invalid option selected for tool mode.")
    sys.exit()

# Append .zip to the file name
zip_file = f"{park_name}.zip"

# Check if the zip file exists
if os.path.exists(zip_file):
    # Create a ZipFile object
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extract all contents to a folder named after the park code
        zip_ref.extractall(f"{park_name}_unzipped")
        print(f"Extracted {zip_file} to the folder {park_name}_unzipped")
else:
    print(f"The file {zip_file} does not exist.")
    sys.exit()


Enter the Park Code: BRCA
Do you want the tool to be based on 1. the current date or 2. a custom date range: 1
Tool mode entered: 1
Tool mode entered: 1
Show 1. all active schedules or 2. all active and future schedules: 2
Current date mode entered: 2
Extracted BRCA.zip to the folder BRCA_unzipped
Current date mode entered: 2
Extracted BRCA.zip to the folder BRCA_unzipped


In [3]:
#Chunk 3: Import Calendar

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the calendar.txt file
calendar_file = os.path.join(unzipped_folder, "calendar.txt")

# Check if the file exists
if os.path.exists(calendar_file):
    # Load the calendar.txt into a pandas DataFrame
    calendar = pd.read_csv(calendar_file)
    print("Loaded calendar.txt into a DataFrame.")
    print(calendar.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {calendar_file} does not exist.")

Loaded calendar.txt into a DataFrame.
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

   start_date  end_date  
0    20240923  20501230  
Loaded calendar.txt into a DataFrame.
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

   start_date  end_date  
0    20240923  20501230  


In [4]:
# Convert start_date and end_date columns to datetime 
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

if tool_mode == '1':
    if current_date_mode == '2':
        # Get the current date as a datetime object
        current_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
        # Filter rows where the current date is between start_date and end_date or end_date is in the future
        active_schedules = calendar[(calendar['start_date'] <= current_date) & (calendar['end_date'] >= current_date) | (calendar['end_date'] > current_date)]
        print(f"Active/Future schedules on {current_date.date()}:")
        print(active_schedules)
    else:
        # Get the current date as a datetime object
        current_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
        # Filter rows where the current date is between start_date and end_date or end_date is in the future
        active_schedules = calendar[(calendar['start_date'] <= current_date) & (calendar['end_date'] >= current_date)] 
        print(f"Active schedules on {current_date.date()}:")
        print(active_schedules)
elif tool_mode == '2':
        # Filter rows where the schedule end_date is between custom_start and custom_end
        # or the schedule overlaps with the custom date range (starts before custom_end and ends after custom_start)
        active_schedules = calendar[(calendar['end_date'] >= custom_start) & (calendar['start_date'] <= custom_end)]
        # Display the filtered results
        print(f"Active schedules between {custom_start.date()} and {custom_end.date()}:")
        print(active_schedules)
else:
        print("ERROR")
        sys.exit()

Active/Future schedules on 2024-10-11:
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

  start_date   end_date  
0 2024-09-23 2050-12-30  
Active/Future schedules on 2024-10-11:
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

  start_date   end_date  
0 2024-09-23 2050-12-30  


In [5]:
#Chunk 5: Add trips file

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the trips.txt file
trips_file = os.path.join(unzipped_folder, "trips.txt")
    
if os.path.exists(trips_file): 
    # Load the trips.txt into a pandas DataFrame
    trips = pd.read_csv(trips_file)
    
    # Check if 'wheelchair_accessible' column exists, if not, create it
    if 'wheelchair_accessible' not in trips.columns:
        user_input_wheelchair = input("Wheelchair Accessible? Yes (1) or No (0): ")
        trips['wheelchair_accessible'] = str(int(user_input_wheelchair))  # Assign user input to every row as a string
    else:
        trips['wheelchair_accessible'] = trips['wheelchair_accessible'].fillna(0)
        trips['wheelchair_accessible'] = trips['wheelchair_accessible'].apply(lambda x: str(int(x)))
        trips['wheelchair_accessible'] = trips['wheelchair_accessible'].replace(0, '')
    
    # Check if 'bikes_allowed' column exists, if not, create it
    if 'bikes_allowed' not in trips.columns:
        user_input_bikes = input("Bikes Allowed? Yes (1) or No (0): ")
        trips['bikes_allowed'] = str(int(user_input_bikes))  # Assign user input to every row as a string
    else:
        trips['bikes_allowed'] = trips['bikes_allowed'].fillna(0)
        trips['bikes_allowed'] = trips['bikes_allowed'].apply(lambda x: str(int(x)))
        trips['bikes_allowed'] = trips['bikes_allowed'].replace(0, '')
    
    print("Loaded trips.txt into a DataFrame.")
    print(trips.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {trips_file} does not exist.")

Wheelchair Accessible? Yes (1) or No (0): 1
Bikes Allowed? Yes (1) or No (0): 1
Loaded trips.txt into a DataFrame.
   route_id  service_id  trip_id  trip_headsign  trip_short_name  \
0        10          59     2807            NaN              NaN   
1        10          59     2808            NaN              NaN   
2        10          59     2809            NaN              NaN   
3        10          59     2810            NaN              NaN   
4        10          59     2811            NaN              NaN   

   direction_id block_id  shape_id wheelchair_accessible bikes_allowed  
0           NaN  1A-0800        10                     1             1  
1           NaN  1A-0900        10                     1             1  
2           NaN  1A-1000        10                     1             1  
3           NaN  1A-1100        10                     1             1  
4           NaN  1A-1200        10                     1             1  
Loaded trips.txt into a DataFrame.
   

In [6]:
#Chunk 6: Filter to active trips

# Filter trips where service_id in trips matches any service_id in active_schedules
active_trips = trips[trips['service_id'].isin(active_schedules['service_id'])]

print("Active trips based on active schedules:")
print(active_trips)

Active trips based on active schedules:
    route_id  service_id  trip_id  trip_headsign  trip_short_name  \
0         10          59     2807            NaN              NaN   
1         10          59     2808            NaN              NaN   
2         10          59     2809            NaN              NaN   
3         10          59     2810            NaN              NaN   
4         10          59     2811            NaN              NaN   
5         10          59     2812            NaN              NaN   
6         10          59     2813            NaN              NaN   
7         10          59     2814            NaN              NaN   
8         10          59     2815            NaN              NaN   
9         10          59     2816            NaN              NaN   
10        10          59     2817            NaN              NaN   
11        10          59     2818            NaN              NaN   
12        10          59     2819            NaN              N

In [7]:
#Chunk 7: Add stop_times 

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the stop_times.txt file
stop_times_file = os.path.join(unzipped_folder, "stop_times.txt")

# Check if the file exists
if os.path.exists(stop_times_file):
    # Load the stop_times.txt into a pandas DataFrame
    stop_times = pd.read_csv(stop_times_file)
    print("Loaded stop_times.txt into a DataFrame.")
    print(stop_times.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {stop_times_file} does not exist.")

Loaded stop_times.txt into a DataFrame.
   trip_id arrival_time departure_time  stop_id  stop_sequence  stop_headsign  \
0     2807     08:00:00       08:00:00        1              1            NaN   
1     2807     08:02:00       08:02:00       12              2            NaN   
2     2807     08:03:00       08:03:00       13              3            NaN   
3     2807     08:04:00       08:04:00       14              4            NaN   
4     2807     08:06:00       08:06:00       15              5            NaN   

   pickup_type  drop_off_type  timepoint  
0            0              0          1  
1            0              0          1  
2            0              0          1  
3            0              0          1  
4            0              0          1  
Loaded stop_times.txt into a DataFrame.
   trip_id arrival_time departure_time  stop_id  stop_sequence  stop_headsign  \
0     2807     08:00:00       08:00:00        1              1            NaN   
1     2807   

In [8]:
#Chunk 8: Fitler out active stop times
active_stop_times = stop_times[stop_times['trip_id'].isin(active_trips['trip_id'])]

print(f"Active stop times DataFrame created with {len(active_stop_times)} rows.")
print(active_stop_times.head())  # Print the first few rows of the active_stop_times DataFrame

Active stop times DataFrame created with 896 rows.
   trip_id arrival_time departure_time  stop_id  stop_sequence  stop_headsign  \
0     2807     08:00:00       08:00:00        1              1            NaN   
1     2807     08:02:00       08:02:00       12              2            NaN   
2     2807     08:03:00       08:03:00       13              3            NaN   
3     2807     08:04:00       08:04:00       14              4            NaN   
4     2807     08:06:00       08:06:00       15              5            NaN   

   pickup_type  drop_off_type  timepoint  
0            0              0          1  
1            0              0          1  
2            0              0          1  
3            0              0          1  
4            0              0          1  
Active stop times DataFrame created with 896 rows.
   trip_id arrival_time departure_time  stop_id  stop_sequence  stop_headsign  \
0     2807     08:00:00       08:00:00        1              1          

In [9]:
#Chunk 9: Add stops

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the stops.txt file
stops_file = os.path.join(unzipped_folder, "stops.txt")

# Check if the file exists
if os.path.exists(stops_file):
    # Load the stops.txt into a pandas DataFrame
    stops = pd.read_csv(stops_file)
    print("Loaded stops.txt into a DataFrame.")
    print(stops.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {stops_file} does not exist.")

Loaded stops.txt into a DataFrame.
   stop_id  stop_code                 stop_name  stop_desc   stop_lat  \
0        1        NaN           Shuttle Station        NaN  37.671111   
1       12        NaN            Old Bryce Town        NaN  37.673567   
2       13        NaN  Bryce Canyon Grand Hotel        NaN  37.674484   
3       14        NaN                Ruby's Inn        NaN  37.673443   
4       15        NaN    Ruby's Campground (SB)        NaN  37.667808   

     stop_lon  stop_url  location_type  parent_station  
0 -112.156920       NaN              0             NaN  
1 -112.156457       NaN              0             NaN  
2 -112.154010       NaN              0             NaN  
3 -112.156700       NaN              0             NaN  
4 -112.158195       NaN              0             NaN  
Loaded stops.txt into a DataFrame.
   stop_id  stop_code                 stop_name  stop_desc   stop_lat  \
0        1        NaN           Shuttle Station        NaN  37.671111   
1  

In [10]:
#Chunk 10: Fitler out active stops
active_stops = stops[stops['stop_id'].isin(active_stop_times['stop_id'])]

print(f"Active stop times DataFrame created with {len(active_stops)} rows.")
print(active_stops.head())  # Print the first few rows of the active_stop_times DataFrame

Active stop times DataFrame created with 16 rows.
   stop_id  stop_code                 stop_name  stop_desc   stop_lat  \
0        1        NaN           Shuttle Station        NaN  37.671111   
1       12        NaN            Old Bryce Town        NaN  37.673567   
2       13        NaN  Bryce Canyon Grand Hotel        NaN  37.674484   
3       14        NaN                Ruby's Inn        NaN  37.673443   
4       15        NaN    Ruby's Campground (SB)        NaN  37.667808   

     stop_lon  stop_url  location_type  parent_station  
0 -112.156920       NaN              0             NaN  
1 -112.156457       NaN              0             NaN  
2 -112.154010       NaN              0             NaN  
3 -112.156700       NaN              0             NaN  
4 -112.158195       NaN              0             NaN  
Active stop times DataFrame created with 16 rows.
   stop_id  stop_code                 stop_name  stop_desc   stop_lat  \
0        1        NaN           Shuttle Statio

In [11]:
# Chunk 11: Add shape files

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the shapes.txt file
shape_file = os.path.join(unzipped_folder, "shapes.txt")

# Check if the file exists
if os.path.exists(shape_file):
    # Load the shapes.txt into a pandas DataFrame
    shapes = pd.read_csv(shape_file)
    print("Loaded shapes.txt into a DataFrame.")
    print(shapes.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {shape_file} does not exist.")

Loaded shapes.txt into a DataFrame.
   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
0        10     37.671110   -112.156930                  1   
1        10     37.671229   -112.156891                  2   
2        10     37.671270   -112.157130                  3   
3        10     37.672390   -112.156860                  4   
4        10     37.672510   -112.156830                  5   

   shape_dist_traveled  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN  
Loaded shapes.txt into a DataFrame.
   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
0        10     37.671110   -112.156930                  1   
1        10     37.671229   -112.156891                  2   
2        10     37.671270   -112.157130                  3   
3        10     37.672390   -112.156860                  4   
4        10     37.672510   -112.156830                  5   

   shape_dist_traveled  
0      

In [12]:
#Chunk 12: Filter to active shapes
active_shapes = shapes[shapes['shape_id'].isin(active_trips['shape_id'])]

print(f"Active shapes DataFrame created with {len(active_shapes)} rows.")
print(active_shapes.head())  # Print the first few rows of the active_shapes DataFrame

Active shapes DataFrame created with 1159 rows.
   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
0        10     37.671110   -112.156930                  1   
1        10     37.671229   -112.156891                  2   
2        10     37.671270   -112.157130                  3   
3        10     37.672390   -112.156860                  4   
4        10     37.672510   -112.156830                  5   

   shape_dist_traveled  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN  
Active shapes DataFrame created with 1159 rows.
   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
0        10     37.671110   -112.156930                  1   
1        10     37.671229   -112.156891                  2   
2        10     37.671270   -112.157130                  3   
3        10     37.672390   -112.156860                  4   
4        10     37.672510   -112.156830                  5   

   shape

In [13]:
# Chunk 13: Add calendar_dates files

# Define the path to the unzipped folder
unzipped_folder = f"{park_name}_unzipped"

# Define the path to the calendar_dates.txt file
cd_file = os.path.join(unzipped_folder, "calendar_dates.txt")

# Check if the file exists
if os.path.exists(cd_file):
    # Load the calendar_dates.txt into a pandas DataFrame
    cdates = pd.read_csv(cd_file)
    print("Loaded calendar_dates.txt into a DataFrame.")
    print(cdates.head())  # Print the first few rows of the DataFrame
else:
    print(f"The file {shape_file} does not exist.")

Loaded calendar_dates.txt into a DataFrame.
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []
Loaded calendar_dates.txt into a DataFrame.
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []


In [14]:
#Chunk 14: Filter to active calendar dates

# Filter trips where service_id in trips matches any service_id in active_schedules
active_cdates = cdates[cdates['service_id'].isin(active_schedules['service_id'])]

print("Active calendar dates based on active schedules:")
print(active_cdates)

Active calendar dates based on active schedules:
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []
Active calendar dates based on active schedules:
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []


In [15]:
#Chunk 15: Add active feed info
active_feed_info = pd.DataFrame({
        'feed_publisher_name': [None],
        'feed_publisher_url': [None],
        'feed_lang': [None],
        'feed_start_date': [None],
        'feed_end_date': [None],
        'feed_version': [None],
        'feed_contact_email': [None],
        'feed_contact_url': [None]
    })
    
# Find the earliest start_date and latest end_date
active_schedules['start_date'] = pd.to_datetime(active_schedules['start_date'], errors='coerce', format='%Y%m%d')
active_schedules['end_date'] = pd.to_datetime(active_schedules['end_date'], errors='coerce', format='%Y%m%d')
earliest_start_date = active_schedules['start_date'].min().strftime('%Y%m%d')
latest_end_date = active_schedules['end_date'].max().strftime('%Y%m%d')

# Get the current date in YYYYMMDD format
current_date = datetime.now().strftime('%Y%m%d')

# Update the active_feed_info DataFrame
active_feed_info['feed_start_date'] = earliest_start_date
active_feed_info['feed_end_date'] = latest_end_date
active_feed_info['feed_version'] = current_date
active_feed_info['feed_publisher_name'] = "NPS " + park_name
active_feed_info['feed_publisher_url'] = "https://www.nps.gov/subjects/developer/gtfs.htm"
active_feed_info['feed_lang'] = "en"
active_feed_info['feed_contact_email'] = "erica.cole@nps.gov"
active_feed_info['feed_contact_url'] = ""
print("Updated feed info based on current date and schedule range:")
print(active_feed_info.head())

Updated feed info based on current date and schedule range:
  feed_publisher_name                               feed_publisher_url  \
0            NPS BRCA  https://www.nps.gov/subjects/developer/gtfs.htm   

  feed_lang feed_start_date feed_end_date feed_version  feed_contact_email  \
0        en        20240923      20501230     20241011  erica.cole@nps.gov   

  feed_contact_url  
0                   
Updated feed info based on current date and schedule range:
  feed_publisher_name                               feed_publisher_url  \
0            NPS BRCA  https://www.nps.gov/subjects/developer/gtfs.htm   

  feed_lang feed_start_date feed_end_date feed_version  feed_contact_email  \
0        en        20240923      20501230     20241011  erica.cole@nps.gov   

  feed_contact_url  
0                   


In [16]:
#Chunk 16: Convert calendar dates back to GTFS format (YYYYMMDD)
active_schedules['start_date'] = active_schedules['start_date'].dt.strftime('%Y%m%d')
active_schedules['end_date'] = active_schedules['end_date'].dt.strftime('%Y%m%d')

print("Converted dates in active_schedules DataFrame:")
print(active_schedules.head())  # Print the first few rows to verify the changes

Converted dates in active_schedules DataFrame:
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

  start_date  end_date  
0   20240923  20501230  
Converted dates in active_schedules DataFrame:
   service_id  monday  tuesday  wednesday  thursday  friday  saturday  sunday  \
0          59       1        1          1         1       1         1       1   

  start_date  end_date  
0   20240923  20501230  


In [17]:
#Chunk 17: Create new archive for the active, filtered GTFS feed

# Define the source and destination folders 
source_folder = f"{park_name}_unzipped"
destination_folder = f"{park_name}_active_unzipped"

# Check if the destination folder exists
if os.path.exists(destination_folder):
    # Delete all files in the destination folder
    for file_name in os.listdir(destination_folder):
        file_path = os.path.join(destination_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print(f"Deleted all files in {destination_folder}.")
else:
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    print(f"Created destination folder: {destination_folder}.")

# Check if the source folder exists
if os.path.exists(source_folder):
    # Transfer all files from source to destination
    for file_name in os.listdir(source_folder):
        # Construct full file path
        full_file_name = os.path.join(source_folder, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, destination_folder)
    
    print(f"All files transferred from {source_folder} to {destination_folder}.")
else:
    print(f"The source folder {source_folder} does not exist.")

Deleted all files in BRCA_active_unzipped.
Deleted all files in BRCA_active_unzipped.
All files transferred from BRCA_unzipped to BRCA_active_unzipped.
All files transferred from BRCA_unzipped to BRCA_active_unzipped.


In [18]:
#Chunk 18: Delete any old files, update to active versions

# Define the paths to the files to delete
calendar_file_path = os.path.join(destination_folder, "calendar.txt")
trips_file_path = os.path.join(destination_folder, "trips.txt")
stop_times_file_path = os.path.join(destination_folder, "stop_times.txt")
shape_file_path = os.path.join(destination_folder, "shapes.txt")
cdate_file_path = os.path.join(destination_folder, "calendar_dates.txt")
stops_file_path = os.path.join(destination_folder, "stops.txt")
feed_info_file_path = os.path.join(destination_folder, "feed_info.txt")

# Delete the old files
if os.path.exists(calendar_file_path):
    os.remove(calendar_file_path)
    print(f"Deleted {calendar_file_path}.")
else:
    print(f"{calendar_file_path} does not exist.")

if os.path.exists(trips_file_path):
    os.remove(trips_file_path)
    print(f"Deleted {trips_file_path}.")
else:
    print(f"{trips_file_path} does not exist.")

if os.path.exists(stop_times_file_path):
    os.remove(stop_times_file_path)
    print(f"Deleted {stop_times_file_path}.")
else:
    print(f"{stop_times_file_path} does not exist.")
    
if os.path.exists(shape_file_path):
    os.remove(shape_file_path)
    print(f"Deleted {shape_file_path}.")
else:
    print(f"{shape_file_path} does not exist.")
    
if os.path.exists(cdate_file_path):
    os.remove(cdate_file_path)
    print(f"Deleted {cdate_file_path}.")
else:
    print(f"{cdate_file_path} does not exist.")

if os.path.exists(stops_file_path):
    os.remove(stops_file_path)
    print(f"Deleted {stops_file_path}.")
else:
    print(f"{stops_file_path} does not exist.")

if os.path.exists(feed_info_file_path):
    os.remove(feed_info_file_path)
    print(f"Deleted {feed_info_file_path}.")
else:
    print(f"{feed_info_file_path} does not exist.")
    
# Save active info to folder
calendar_file = os.path.join(destination_folder, "calendar.txt")
trips_file = os.path.join(destination_folder, "trips.txt")
stop_times_file = os.path.join(destination_folder, "stop_times.txt")
shape_file = os.path.join(destination_folder, "shapes.txt")
cdate_file = os.path.join(destination_folder, "calendar_dates.txt")
stops_file = os.path.join(destination_folder, "stops.txt")
feed_info_file = os.path.join(destination_folder, "feed_info.txt")

active_schedules.to_csv(calendar_file, sep=',', index=False)
active_trips.to_csv(trips_file, sep=',', index=False)
active_stop_times.to_csv(stop_times_file, sep=',', index=False)
active_shapes.to_csv(shape_file, sep=',', index=False)
active_cdates.to_csv(cdate_file, sep=',', index=False)
active_stops.to_csv(stops_file, sep=',', index=False)
active_feed_info.to_csv(feed_info_file, sep=',', index=False)

print(f"Saved active schedules to {calendar_file}.")
print(f"Saved active trips to {trips_file}.")
print(f"Saved active stop times to {stop_times_file}.")
print(f"Saved active shapes to {shape_file}.")
print(f"Saved active calendar dates to {cdate_file}.")
print(f"Saved active stops to {stops_file}.")
print(f"Saved active feed info dates to {feed_info_file}.")

Deleted BRCA_active_unzipped\calendar.txt.
Deleted BRCA_active_unzipped\calendar.txt.
Deleted BRCA_active_unzipped\trips.txt.
Deleted BRCA_active_unzipped\stop_times.txt.
Deleted BRCA_active_unzipped\shapes.txt.
Deleted BRCA_active_unzipped\calendar_dates.txt.
Deleted BRCA_active_unzipped\stops.txt.
Deleted BRCA_active_unzipped\feed_info.txt.
Deleted BRCA_active_unzipped\trips.txt.
Deleted BRCA_active_unzipped\stop_times.txt.
Deleted BRCA_active_unzipped\shapes.txt.
Deleted BRCA_active_unzipped\calendar_dates.txt.
Deleted BRCA_active_unzipped\stops.txt.
Deleted BRCA_active_unzipped\feed_info.txt.
Saved active schedules to BRCA_active_unzipped\calendar.txt.
Saved active trips to BRCA_active_unzipped\trips.txt.
Saved active stop times to BRCA_active_unzipped\stop_times.txt.
Saved active shapes to BRCA_active_unzipped\shapes.txt.
Saved active calendar dates to BRCA_active_unzipped\calendar_dates.txt.
Saved active stops to BRCA_active_unzipped\stops.txt.
Saved active feed info dates to BRC

In [19]:
#Chunk 19: Zip it all back up

# Define the path to the active unzipped folder and the zip file name
active_folder = destination_folder
zip_file_name = f"{park_name}_active"

# Create a zip file from the active folder
shutil.make_archive(zip_file_name, 'zip', active_folder)

print(f"Created zip file: {zip_file_name}.zip")

Created zip file: BRCA_active.zip
Created zip file: BRCA_active.zip
